https://adventofcode.com/2021/day/23

Not very good. Should have tried just using a list of chars as data structure, forget about classes to model amphipods, halls, and rooms.

In [1]:
from dataclasses import dataclass
from heapq import heappush, heappop
from collections import deque

from toolz import partition

In [2]:
data = """\
#############
#...........#
###D#B#A#C###
  #C#A#D#B#
  #########
"""

datastate = '.......DCBAADCB'


In [3]:
testdata = """\
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
"""

teststate = '.......BACDBCDA'

In [4]:
finalstate = '.......AABBCCDD'

In [17]:
amphipod_energy = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

class Amphipod:
    def __init__(self, label, node=None):
        self.label = label
        self.energy = amphipod_energy[label]
        self.node = node
        
    def __repr__(self):
        return '<A ' + self.label + ' ' + str(self.node) + ' >'

    def canstart(self, blockers=None):
        blockers = blockers or set()
        
        if not any(x for x, _ in self.node.neighbors if x not in blockers and x.canreceive(self)):
            return False
        
        if isinstance(self.node, Hall):
            # Can't start without clear path to home
            for node in self.node.iter_reachable():
                if node.label == self.label:
                    return node.canreceive(self)
        
        # node is a Room
        return self.node.cangive() and self.node.stack[-1] is self
        
    
    

In [77]:
class Hall:
    def __init__(self):
        self.label = '.'
        self.neighbors = [] 
        self.occupant = None
        
    def getstate(self):
        return '.' if self.occupant is None else self.occupant.label
    
    def give(self):
        occupant = self.occupant
        if occupant is None:
            raise IndexError('Hall node is unoccupied: %s' % self)
        self.occupant = None
        return (occupant, 0)

    def receive(self, ap):
        if self.occupant is not None:
            raise IndexError('Hall node is already occupied: %s' % self)
        self.occupant = ap
        return 0
    
    def canreceive(self, ap):
        return self.occupant is None

    def iter_reachable(self, blockers=None, visited=None):
        if visited is None:
            visited = set()
        if blockers is None:
            blockers = set()
        for nabe, weight in self.neighbors:
            if nabe in visited:
                continue
            if nabe in blockers:
                continue
            visited.add(nabe)
            yield nabe
            yield from nabe.iter_reachable(blockers=blockers, visited=visited)
    
    def __repr__(self):
        return '<H>'
    
    
class Room:
    def __init__(self, label, size=2):
        self.label = label
        self.size = size
        self.stack = []
        self.neighbors = []
    
    def getstate(self):
        return ''.join(ap.label for ap in self.stack)[::-1].rjust(self.size, '.')
    
    def give(self):
        if not self.stack:
            raise IndexError('Room node has empty stack: %s' % self)
        extrasteps = self.size - len(self.stack)
        return self.stack.pop(), extrasteps
    
    def receive(self, ap):
        if len(self.stack) == self.size:
            raise IndexError('Room node has full stack: %s' % self)
        self.stack.append(ap)
        return self.size - len(self.stack)
        
    def cangive(self):
        return any(x for x in self.stack if x.label != self.label)
    
    def canreceive(self, ap):
        return (self.label == ap.label
                and len(self.stack) < self.size
                and not any(x for x in self.stack if x.label != self.label))
    
    def iter_reachable(self, *args, **kwargs):
        return iter([])
    
    def __repr__(self):
        return '<R ' + self.label + ' ' + str(self.stack) + '>'

In [7]:
def construct_burrow(roomsize=2):
    hallway = [Hall() for i in range(7)]
    for (i1, i2) in [(0, 1), (5, 6)]:
        hallway[i1].neighbors.append((hallway[i2], 1))
        hallway[i2].neighbors.append((hallway[i1], 1))
    for (i1, i2) in [(1, 2), (2, 3), (3, 4), (4, 5)]:
        hallway[i1].neighbors.append((hallway[i2], 2))
        hallway[i2].neighbors.append((hallway[i1], 2))
    
    rooms = []
    for label, pos in [('A', 1), ('B', 2), ('C', 3), ('D', 4)]:
        rs = Room(label=label, size=roomsize)
        rooms.append(rs)
        hallway[pos].neighbors.append((rs, 2))
        rs.neighbors.append((hallway[pos], 2))
        hallway[pos+1].neighbors.append((rs, 2))
        rs.neighbors.append((hallway[pos+1], 2))
    
    return hallway + rooms

In [8]:
def getstate(burrow):
    return ''.join(x.getstate() for x in burrow)

In [69]:
def setstate(burrow, state):
    aps = []

    for c, hall in zip(state[:7], burrow[:7]):
        if c == '.':
            hall.occupant = None
        else:
            ap = Amphipod(label=c, node=hall)
            hall.occupant = ap
            aps.append(ap)

    roomstr = state[7:]
    roomsize = len(roomstr) // 4
    for s, room in zip((''.join(x) for x in partition(roomsize, roomstr)), burrow[7:]):
        room.stack = []
        for aplabel in s[::-1]:
            if aplabel != '.':
                ap = Amphipod(label=aplabel, node=room)
                room.stack.append(ap)
                aps.append(ap)

        
    return aps

In [42]:
burrow = construct_burrow(roomsize=2)

In [43]:
burrow

[<H>, <H>, <H>, <H>, <H>, <H>, <H>, <R A []>, <R B []>, <R C []>, <R D []>]

In [44]:
aps = setstate(burrow, teststate)

In [45]:
getstate(burrow)

'.......BACDBCDA'

In [46]:
burrow

[<H>,
 <H>,
 <H>,
 <H>,
 <H>,
 <H>,
 <H>,
 <R A [<A A <R A [...]> >, <A B <R A [...]> >]>,
 <R B [<A D <R B [...]> >, <A C <R B [...]> >]>,
 <R C [<A C <R C [...]> >, <A B <R C [...]> >]>,
 <R D [<A A <R D [...]> >, <A D <R D [...]> >]>]

In [47]:
burrow[-1].stack

[<A A <R D [<A A <R D [...]> >, <A D <R D [...]> >]> >,
 <A D <R D [<A A <R D [...]> >, <A D <R D [...]> >]> >]

In [50]:
burrow[-1].give()

(<A D <R D [<A A <R D [...]> >]> >, 0)

In [52]:
burrow[-1].stack

[<A A <R D [<A A <R D [...]> >]> >]

In [92]:
def dijkstra(burrow, initialstate, finalstate):
    frontier = []
    visited = {}
    heappush(frontier, (0, initialstate))
    visited[initialstate] = 0
    
    while visited:
        cost, state = heappop(frontier)
        if state == finalstate:
            return cost
        aps = setstate(burrow, state)
        for i in range(len(aps)):
            ap = aps[i]
            blockers = {x.node for x in aps if x is not ap and isinstance(x.node, Hall)}
            if not ap.canstart(blockers):
                continue
            for subcost in subbfs(ap, blockers):
                newcost = cost + subcost
                newstate = getstate(burrow)
                if newstate not in visited or visited[newstate] > newcost:
                    heappush(frontier, (newcost, newstate))
                    visited[newstate] = newcost
            aps = setstate(burrow, state)
            
            
def subbfs(ap, blockers):
    q = deque([])
    v = set()
    energy = ap.energy
    node = ap.node
    ap, _ = node.give()

#     print(ap)
#     print(node)
#     print()
    
    for nabe, steps in node.neighbors:
        q.append((node, nabe, steps))
#     print(q)
#     print()
    while q:
        node, nabe, steps = q.popleft()
        if (node, nabe) in v or nabe in blockers or not nabe.canreceive(ap):
            continue
        v.add((node, nabe))
        ap.node = node
        node.receive(ap)
        ap, extra = node.give()
        steps += extra
        if not nabe.canreceive(ap):
            print('!!')
            print(ap)
            print(nabe)
            print()
        try:
            steps += nabe.receive(ap)
        except:
            print('!')
            print(ap)
            print(node)
            print(nabe)
            print(getattr(nabe, 'stack'))
            raise
        ap.node = nabe
        yield steps * energy
        if ap.label == nabe.label:
            break
        node = nabe.give()
        for nabe2, steps2 in nabe.neighbors:
            q.append((nabe, nabe2, steps+steps2))


In [93]:
%%time
burrow = construct_burrow(roomsize=2)
dijkstra(burrow, teststate, finalstate)

CPU times: user 4.12 s, sys: 0 ns, total: 4.12 s
Wall time: 4.12 s


12521

In [94]:
%%time
burrow = construct_burrow(roomsize=2)
dijkstra(burrow, datastate, finalstate)

CPU times: user 18 s, sys: 3.21 ms, total: 18 s
Wall time: 18 s


15538

# Part 2

In [88]:
datastate

'.......DCBAADCB'

In [95]:
teststate_2 = '.......BDDACCBDBBACDACA'
datastate_2 = '.......DDDCBCBAABADCACB'
finalstate_2 = '.......AAAABBBBCCCCDDDD'

In [96]:
%%time
# Should be 44169
burrow_2 = construct_burrow(roomsize=4)
dijkstra(burrow_2, teststate_2, finalstate_2)

CPU times: user 15 s, sys: 3.85 ms, total: 15 s
Wall time: 15 s


43289

In [98]:
%%time
# Should be 47258
burrow_2 = construct_burrow(roomsize=4)
dijkstra(burrow_2, datastate_2, finalstate_2)

CPU times: user 14 s, sys: 0 ns, total: 14 s
Wall time: 14 s


47258